In [1]:
# 1） 如何用pytorch 自动计算梯度并使用梯度下降法进行寻找最优解
# 2） 使用numpy如何计算梯度

In [33]:
import torch

x = torch.randn(3,requires_grad = True)
print(x)
y  = x+2 
print(y)
y1 = x*2
print(y1)
y2 = x*x*2

print(y2)
# 如果我们需要计算梯度的话，只需要在参数中使用 requires_grad = True 
# 并且可以看到如果使用的是加法，那么梯度记录的是grad_fn=<AddBackward0>
# 如果使用的是乘法，那么梯度记录的是grad_fn=<MulBackward0>


tensor([ 1.4249,  1.5390, -1.2503], requires_grad=True)
tensor([3.4249, 3.5390, 0.7497], grad_fn=<AddBackward0>)
tensor([ 2.8498,  3.0780, -2.5006], grad_fn=<MulBackward0>)
tensor([4.0607, 4.7371, 3.1265], grad_fn=<MulBackward0>)


In [27]:
print(x)
y3 = x.mean()
y3.backward() # dz/dx
print(x.grad)
# 这里我们调用的是自带的mean方法，所以可以直接使用要y3.backward()
# 如果不是一些内置的函数，那么需要给他一个取值，才可以使用backward，eg上述的y2的函数
y4 = torch.tensor([0.1,1.0,0.001],dtype = torch.float32)
y2.backward(y4)
print(x.grad)

tensor([-2.2168,  0.1898,  1.4724], requires_grad=True)
tensor([0.3333, 0.3333, 0.3333])
tensor([-0.5534,  1.0924,  0.3392])


In [38]:
#   如何取消tracing 梯度

x = torch.randn(3,requires_grad = True)
print(x)
# 方法一 ：设置requires_grad_(False)
x.requires_grad_(False)
print(x)
# 可以看到x已经不再带有requires_grad = True,这是直接修改x的
# 方法二 ： x.detach()
z = x.detach()
print(z)
# 可以看到z已经不再带有requires_grad = True,这里不是直接修改x的
# 方法三 ： with torch.no_grad():
with torch.no_grad():
    y = x+2
    print(y)
# 可以看到这里y已经不再带有requires_grad = True

# 再来验证一下如果不使用 with torch.no_grad():,查看一下y 还是会tracing 梯度的
x = torch.randn(3,requires_grad = True)
y = x+2
print(y)


tensor([-0.2977,  0.9836, -1.7483], requires_grad=True)
tensor([-0.2977,  0.9836, -1.7483])
tensor([-0.2977,  0.9836, -1.7483])
tensor([1.7023, 2.9836, 0.2517])
tensor([1.2402, 3.0584, 1.7641], grad_fn=<AddBackward0>)


In [43]:
# 需要注意的是梯度迭代相加的问题，可以看到在每循环一次梯度都会增加

weights = torch.ones(4,requires_grad=True)
for epoch in range(1):
    model_outputs = (weights * 3).sum()
    model_outputs.backward()
    print(weights.grad)
print('==============')

weights = torch.ones(4,requires_grad=True)
for epoch in range(2):
    model_outputs = (weights * 3).sum()
    model_outputs.backward()
    print(weights.grad)
print('==============')

weights = torch.ones(4,requires_grad=True)
for epoch in range(3):
    model_outputs = (weights * 3).sum()
    model_outputs.backward()
    print(weights.grad)
print('==============')

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([9., 9., 9., 9.])


In [9]:
# 为了避免上面的情况，我们需要在每次计算完梯度之后把梯度清空,这样就会看到我们的提速就是正确的了
import torch
weights = torch.ones(4,requires_grad=True)
for epoch in range(1):
    model_outputs = (weights * 3).sum()
    model_outputs.backward()
    print(weights.grad)   
print('==============')

weights = torch.ones(4,requires_grad=True)
for epoch in range(2):
    model_outputs = (weights * 3).sum()
    model_outputs.backward()
    print(weights.grad)
    weights.grad.zero_()
   
print('==============')

weights = torch.ones(4,requires_grad=True)
for epoch in range(3):
    model_outputs = (weights * 3).sum()
    model_outputs.backward()
    print(weights.grad)
    weights.grad.zero_()
print('==============')

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
